In [ ]:
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(8):
        if (cur / "src").exists() and (cur / "README.md").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    raise RuntimeError("Repo root not found. Start Jupyter from the repo or adjust the search.")

ROOT = find_repo_root(Path.cwd())
print("ROOT:", ROOT)


In [ ]:
!python {ROOT}/src/scripts/1-option-chain-build-historic-dataset-v1.0.py \
  --out-dir {ROOT}/src/data/raw \
  --out-name options_chain_dataset.csv \
  --drops-name option_chain_historic_dataset_drops.csv \
  --tickers NVDA,AAPL,GOOGL,MSFT,META,AMZN,PLTR \
  --start 2023-06-01 --end 2023-06-30 \
  --strike-range 0 \
  --min-band-strikes 6 \
  --min-band-prn-strikes 4 \
  --min-prn-train 0.08 --max-prn-train 0.92 \
  --prefer-bidask \
  --min-chain-used-hard 0 \
  --write-drops \
  --stock-source yfinance \
  --no-cache


In [ ]:
!python {ROOT}/src/scripts/2-calibrate-logit-model-v1.5.py \
  --csv {ROOT}/src/data/raw/options_chain_dataset.csv \
  --out-dir {ROOT}/src/data/models/v1-5-ticker-intercepts \
  --features x_logit_prn,T_days,abs_log_m,rv20 \
  --calibrate platt \
  --test-weeks 20 \
  --val-windows 4 \
  --val-window-weeks 10 \
  --fit-weight-renorm mean1 \
  --train-decay-half-life-weeks 52 \
  --schema {ROOT}/config/options_dataset_schema.json \
  --feature-allowlist {ROOT}/config/feature_allowlist.json
